# **Example PSI Model Synthesis Notebook**

Tom Schad 

----

- This notebook does an example synthesis of the Global corona using a Predictive Sciences MHD model. 


----

In [13]:
import pycelp 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import os
os.environ["XUVTOP"] = '/usr/local/ssw/packages/chianti/dbase/'
import psi

## Create an instance of the psi.Model class to load and interact with PSI coronal model data 

- Models used here were downloaded using this website from [Predictive Sciences](https://www.predsci.com/hmi/data_access.php)
- We picked the date of the US total solar eclipse (21 Aug 2017) using the med-cor-thermo2 model for this demonstration

In [14]:
corona = psi.Model('/home/tschad/Dropbox/psiData_21aug2017_12UTC_med-cor-thermo2/corona/')

In [15]:
corona.rlen.shape,corona.blen.shape,corona.b_localinc.shape,corona.lats.shape,corona.lons.shape

((181, 100, 150), (181, 100, 150), (181, 100, 150), (100,), (181,))

In [16]:
fig,ax = plt.subplots(2,1,figsize = (5,6))
ax = ax.flatten()
ax[0].imshow(corona.blen[:,:,12].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
ax[1].imshow(corona.b_localinc[:,:,12].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Initialize the pyCELP model of the line or lines to be synthesized 

- Here we start with Fe XIII 1074 nm and keep the model fairly "small" at 50 levels.  The errors incurred by using reduced numbers of atomic levels is addressed in [Schad & Dima (2020)](https://rdcu.be/b5J2X) 


In [17]:
fe13 = pycelp.Ion('fe_13',nlevels = 50)

 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.elvlc
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.wgfa
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.scups
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.psplups
 using default abundances: /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 reading:  /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 testing default file: /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 reading:  /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 setting up electron collision rate factors
 setting up proton  collision rate factors
 setting up non-dipole radiative rate factors
 getting non-dipole rate factors
 setting up dipole radiative rate factors


In [18]:
# should I parallelize these calculations right away? 

In [7]:
corona.ne.shape

(181, 100, 150)

In [8]:
wvair = 10747. 

ii,jj,kk = 45,15,10
for ii in range(0,181,10): 
    print(ii)
    for jj in range(0,100,10): 
        for rr in range(0,150,10): 
            edens = corona.ne[ii,jj,kk]
            etemp = corona.temp[ii,jj,kk]
            ht = corona.rlen[ii,jj,kk]-1
            thetab = np.rad2deg(corona.b_localinc[ii,jj,kk])
            
            fe13.calc_rho_sym(edens,etemp,ht, thetab)
            
            upper_lev_rho00 = fe13.get_upper_level_rho00(wvair)
            upper_lev_alignment = fe13.get_upper_level_alignment(wvair)
            total_ion_population = fe13.totn


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180


In [49]:
## LOS integration of polarized emission coefficients 

In [39]:
## generating polarized spectra